In [1]:
import os,time,sys,subprocess,warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import tensorflow as tf
from pathlib import Path
from tensorflow import keras
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_dark"
tf.get_logger().setLevel("ERROR")

# Creating the Training Dataset

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
shake = keras.utils.get_file("shakes.txt",'https://homl.info/shakespeare')
with open(shake) as f:
    shakes = f.read()

In [4]:
print(shakes[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


- encode every character to integer
  - one option to create custom preprocessing layer
  - second is to use keras tokenizer class

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakes)
tokenizer.texts_to_sequences(["Hello this is mario"])
print("".join(tokenizer.sequences_to_texts(np.random.randint(1,50,size=[1,50]))))
max_id = len(tokenizer.word_index)
total_size = tokenizer.document_count
encoded = np.array(tokenizer.texts_to_sequences(shakes))-1
encoded[:5]

. a 3 z s z $ y y i i & & 3 
 n t t   z & ? e q p y o p z p ! - j b o


array([[19],
       [ 5],
       [ 8],
       [ 7],
       [ 2]])

In [6]:
token_layer = keras.layers.TextVectorization(split="character",standardize="lower")
token_layer.adapt([shakes])
encoded = token_layer([shakes])[0]

In [72]:
n_tokens = token_layer.vocabulary_size()-2

In [7]:
train_size = total_size * 90//100
valid_size = total_size - train_size
print(f"train size = {train_size} and valid size = {valid_size}")

train size = 1003854 and valid size = 111540


In [8]:
n_steps = 100
batch_size = 32
shuffle_size = int(1e4)

In [9]:
tf.config.list_logical_devices()
strategy = tf.distribute.OneDeviceStrategy(device="/device:GPU:0")

In [10]:
def create_ds(slices,seq_len=100,batch_size=32,shuffle_size=10000,shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.window(seq_len+1,shift=1,drop_remainder=True).flat_map(lambda x: x.batch(seq_len+1))
    if shuffle:
        ds = ds.shuffle(shuffle_size)
    ds = ds.batch(batch_size)
    ds = ds.map(lambda x: (x[:,:-1],x[:,1:]))
    return ds.prefetch(1)

In [11]:
train_ds = create_ds(encoded[:train_size],shuffle=True)
valid_ds = create_ds(encoded[train_size:train_size+80_000])
test_ds = create_ds(encoded[train_size+80_000:])

In [12]:
# with strategy.scope():
#     model = keras.Sequential()
#     model.add(keras.layers.Embedding(input_dim=max_id,output_dim=16))
#     model.add(keras.layers.GRU(128,return_sequences=True))
#     model.add(keras.layers.Dense(max_id,"softmax"))
#     model.compile(
#         loss="sparse_categorical_crossentropy",
#         optimizer="nadam",
#         metrics=["accuracy"]
#     )
# checkp = keras.callbacks.ModelCheckpoint("shakes_model",monitor="val_accuracy",save_best_only=True)
# model.fit(train_ds,validation_data=valid_ds,epochs=10,callbacks=[checkp])
'''This will take about an hour so skipping and downloading the model'''

'This will take about an hour so skipping and downloading the model'

In [13]:
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = keras.utils.get_file("shakespeare_model.tgz",origin=url,extract=True)
model_path = Path(path).with_name("shakespeare_model")
shakes_model = keras.models.load_model(model_path)

In [14]:
phrase = "To be or not to b"
tf.argmax(shakes_model.predict([phrase])[0,-1])+2

1/1 [==============================] - 2s 2s/step


<tf.Tensor: shape=(), dtype=int64, numpy=6>

In [15]:
def generate_poem(phrase,chars=50,temperature=1):
    for i in range(chars):
        logits = tf.math.log(shakes_model.predict([phrase],verbose=0)[0,-1:])
        rescaled_logits = logits/temperature
        char_ids = tf.reshape(tf.random.categorical(rescaled_logits,num_samples=1),shape=[])
        phrase += token_layer.get_vocabulary()[char_ids+2]

    print(phrase)
tf.random.set_seed(42)
generate_poem("To be or not to b",temperature=0.01)

To be or not to b
ononrenaiceai
 tceeay:oio:arstagestneevefrsttcaee


# Stateful RNN

In [82]:
def create_stateful_ds(slices,seq_len=100):
    ds = tf.data.Dataset.from_tensor_slices(slices).window(seq_len+1,shift=seq_len,drop_remainder=True).flat_map(lambda x: x.batch(seq_len+1))
    ds = ds.batch(1)
    ds = ds.map(lambda x: (x[:,:-1],x[:,1:])).prefetch(1)
    return ds

In [87]:
for i in create_stateful_ds(encoded[:train_size]).take(1):
    print(i[0].shape,i[1].shape)

(1, 100) (1, 100)


In [ ]:
train_ds = create_stateful_ds(encoded[:train_size])
valid_ds = create_stateful_ds(encoded[train_size:train_size+70_000])

In [74]:
stateful_shakes = keras.Sequential([
    keras.layers.Embedding(input_dim=n_tokens,output_dim=16,batch_input_shape=[1,None]),
    keras.layers.GRU(128,return_sequences=True,stateful=True),
    keras.layers.Dense(n_tokens,activation="softmax")
])
stateful_shakes.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)
class ResetStates(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()

In [ ]:
history = stateful_shakes.fit()